In [ ]:
library(Seurat)
library(Matrix)
library(reticulate)
library(glmpca)
library(peakRAM)
library(scry)
library(SingleCellExperiment)
library(glmGamPoi)
library(harmony)
library(reticulate)

# Load data

In [4]:
sparse <- import('scipy.sparse')
mtx <- sparse$load_npz('/data01/hanbin973/fastrna_paper/datasets/mouse_atlas/GSE119945_gene_count_sub.npz')
meta <- read.csv('/data01/hanbin973/fastrna_paper/datasets/mouse_atlas/cell_annotate_sub.csv')

In [5]:
colnames(mtx) <- rownames(meta)
rownames(mtx) <- 1:nrow(mtx)
covmat <- model.matrix(as.formula('~factor(embryo_id)'), meta)
covmat <- covmat[,2:ncol(covmat)]
meta[,paste0('batch_',1:ncol(covmat))] <- covmat

# sctransform

In [6]:
# Including batch variable exhuasts the system memory
benchmark_seurat_sct<- function(obj){
    # create array to store feature variance
    var_feature <- rep(0,nrow(obj))
    names(var_feature) <- 1:nrow(obj)
    
    # run sctransform
    obj <- SCTransform(obj, variable.features.n=2000,  
                       method="glmGamPoi")
    
    # run pca
    obj <- RunPCA(obj, verbose=FALSE)
    
    # save fesult
    write.csv(Embeddings(obj), 'pca_coord/mouse_organogenesis/pca_sct.csv')
    var_feature[rownames(HVFInfo(obj))] <- HVFInfo(obj, 'sct')$residual_variance
    write.csv(var_feature,'feature_var/mouse_organogenesis/var_sct.csv')
    obj
}

In [ ]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_sct(obj))

In [ ]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

# LogNorm

In [6]:
benchmark_seurat_log <- function(obj){
    obj <- NormalizeData(obj)
    obj <- FindVariableFeatures(obj, verbose=FALSE, nfeatures=2000)
    obj <- ScaleData(obj, verbose=FALSE)
    obj <- RunPCA(obj, verbose=FALSE)
    write.csv(Embeddings(obj), 'pca_coord/mouse_organogenesis/pca_log.csv')
    write.csv(HVFInfo(obj)$variance.standardized, 'feature_var/mouse_organogenesis/var_log.csv')
    obj
}

In [8]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_log(obj))

In [9]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,156.251,11497
